Here, you develop the E+M Runge-Kuttas. (You want to be able to graph two functions at a time. The plotting functions can handle it, but the other functions aren't built for it specifically because your cases are simple enough that the answer for the B field is proportional to the E Field.)

In [1]:
import os.path
from scipy import *
import numpy as np
from numpy import *
from numpy import linalg as LA
from scipy import linalg as LA2
import sympy as sympy
import sys as sys
import time
import matplotlib.pyplot as plt
import itertools as it
from IPython.core.display import HTML


sys.path.append('/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/')
from Modules import BasicTools as BT
from Modules import WaveTools as WT
from Modules import PlotTools as PT
from Modules import FFTTools as FFTT
from Modules import OperatorTools as OT
from Modules import GridTransferTools as GTT
from Modules import TestTools as TT
from Modules import SolverTools as ST
from Modules import WaveformTools as WFT

display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth = 10000, threshold = 100000)

In [2]:
nh_min = 64
refRatio = 2
CFL = np.sqrt(0.5)
ntC = 50
nt = int(2*ntC)
x_0 = 0.
x_1 = 0.5
Hans = False

deriv = 'U'
RK = 1
order = 1

L = 1
locs = [x_1]
epsilons = [1, 0.5] # [1, 1]
mus = [1, 0.5] # [1, 1]

func = WFT.Gauss # SquareWave
plotCont = True
enlarge = False

In [5]:
omegaAMR = BT.Grid(nh_min)
finehalf = list(np.arange(int(nh_min / refRatio)))# + int(nh / refRatio))
omegaAMR.AddPatch(refRatio, finehalf)
degFreed = omegaAMR.degFreed
nh_max = omegaAMR.nh_max

omegaF = BT.Grid(nh_max)
omegaC = BT.Grid(nh_min)

In [6]:
physicsAMR = BT.PhysProps(omegaAMR, epsilons, mus, locs, L)
c = physicsAMR.cVec
cMatAMR = physicsAMR.cMat

physicsC = BT.PhysProps(omegaC, epsilons, mus, locs, L)
cC = physicsC.cVec
cMatC = physicsC.cMat

physicsF = BT.PhysProps(omegaF, epsilons, mus, locs, L)
cF = physicsF.cVec
cMatF = physicsF.cMat

cVecC = physicsC.cVec
c1 = cVecC[0]
c2 = cVecC[-1]

distance is
 [ 0.5        0.4921875  0.484375   0.4765625  0.46875    0.4609375  0.453125   0.4453125  0.4375     0.4296875  0.421875   0.4140625  0.40625    0.3984375  0.390625   0.3828125  0.375      0.3671875  0.359375   0.3515625  0.34375    0.3359375  0.328125   0.3203125  0.3125     0.3046875  0.296875   0.2890625  0.28125    0.2734375  0.265625   0.2578125  0.25       0.2421875  0.234375   0.2265625  0.21875    0.2109375  0.203125   0.1953125  0.1875     0.1796875  0.171875   0.1640625  0.15625    0.1484375  0.140625   0.1328125  0.125      0.1171875  0.109375   0.1015625  0.09375    0.0859375  0.078125   0.0703125  0.0625     0.0546875  0.046875   0.0390625  0.03125    0.0234375  0.015625   0.0078125  0.        -0.015625  -0.03125   -0.046875  -0.0625    -0.078125  -0.09375   -0.109375  -0.125     -0.140625  -0.15625   -0.171875  -0.1875    -0.203125  -0.21875   -0.234375  -0.25      -0.265625  -0.28125   -0.296875  -0.3125    -0.328125  -0.34375   -0.359375  -0.375     -0.3906

In [ ]:
# mu, sigma = WFT.GaussParams(x_0, x_1)
mu = 0.25
sigma = 1. / 32.
args = [mu, sigma]